# Dataset: CFGNN to GEM

In [1]:
import pickle
import torch
import numpy as np

## CFGNN

In [2]:
path = "/home/graphAttack/armagaan/code/rough/small_amazon/small_amazon.pickle"
with open(path, "rb") as file:
    data = pickle.load(file)

In [3]:
data.keys()

dict_keys(['adj', 'feat', 'labels', 'train_idx', 'test_idx'])

In [4]:
for key, val in data.items():
    print(key, ":", type(val), end=", ")
    try:
        print(val.shape)
    except:
        print(len(val))

adj : <class 'numpy.ndarray'>, (1, 7650, 7650)
feat : <class 'numpy.ndarray'>, (1, 7650, 745)
labels : <class 'numpy.ndarray'>, (1, 7650)
train_idx : <class 'list'>, 305
test_idx : <class 'list'>, 300


## GEM

In [5]:
# gem style dataset
syn5 = torch.load("../data/syn5/eval_as_eval.pt")

In [6]:
syn5.keys()

dict_keys(['epoch', 'model_type', 'optimizer', 'model_state', 'optimizer_state', 'cg'])

In [7]:
syn5["model_state"].keys()

odict_keys(['gc1.weight', 'gc1.bias', 'gc2.weight', 'gc2.bias', 'gc3.weight', 'gc3.bias', 'lin.weight', 'lin.bias'])

In [8]:
syn5["cg"].keys()

dict_keys(['adj', 'feat', 'train_idx', 'test_idx', 'pred', 'label'])

In [9]:
for key, val in syn5["cg"].items():
    print(key, ":", type(val), end=", ")
    try:
        print(val.shape)
    except:
        print(len(val))

adj : <class 'numpy.ndarray'>, (1, 1231, 1231)
feat : <class 'numpy.ndarray'>, (1, 1231, 10)
train_idx : <class 'list'>, 1102
test_idx : <class 'list'>, 129
pred : <class 'numpy.ndarray'>, (1, 1231, 2)
label : <class 'numpy.ndarray'>, (1, 1231)


## New Checkpoint

In [10]:
adj = data["adj"]
feat = data["feat"]
label = data["labels"]
# eval as eval
train_idx = data["train_idx"]
test_idx = data["test_idx"]

In [11]:
from copy import deepcopy
new_ckpt = deepcopy(syn5)

new_ckpt["cg"]["adj"] = adj
new_ckpt["cg"]["feat"] = feat
new_ckpt["cg"]["train_idx"] = train_idx
new_ckpt["cg"]["test_idx"] = test_idx
new_ckpt["cg"]["label"] = label

In [12]:
import sys
sys.path.append("../gnnexp/")
from models import GCNSynthetic

model = GCNSynthetic(
    nfeat=feat.shape[-1],
    nhid=128, # change according to blackbox
    nout=128, # change according to blackbox
    nclass=8, # change according to dataset
    dropout=0.2,
    device="cpu"
)

weights = torch.load("../cfgnn_model_weights/gcn_3layer_small_amazon.pt")
model.load_state_dict(weights)

<All keys matched successfully>

In [13]:
feat = torch.from_numpy(feat).float()
adj = torch.from_numpy(adj).float()
label = torch.tensor(label).long()
model.eval()
pred = model(feat, adj)
pred = pred.detach().numpy()

In [14]:
new_ckpt["cg"]["pred"] = pred
new_ckpt["model_state"] = weights

In [15]:
torch.save(obj=new_ckpt, f="../data/small_amazon/eval_as_eval.pt")